In [81]:
import pandas as pd
import numpy as np
import os
from itertools import combinations
import datacompy
folder_path=r'C:\Users\hoa.nd\Desktop\WAP\compare\Compare report'

In [82]:

try:
  df_cognos = pd.read_pickle('df_cognos.pkl')
  df_looker = pd.read_pickle('df_looker.pkl')
except FileNotFoundError:
    # If pickle files are not found, read Excel files
  df_cognos = pd.read_excel(r'C:\Users\hoa.nd\Desktop\WAP\compare\Compare report\AR History Package-Mthly Invoiced Amt - 1.xlsx',sheet_name='By Region',dtype=str)
  df_looker = pd.read_excel(r'C:\Users\hoa.nd\Desktop\WAP\compare\Compare report\AR-region-looker.xlsx', dtype=str)
  
  # Save DataFrames to pickle files
  df_cognos.to_pickle('df_cognos.pkl')
  df_looker.to_pickle('df_looker.pkl')

COGNOS_TO_LOOKER = {
  'Invoice Amount (USD)':'S/A OPUS Month'
}
df_cognos.rename(columns=COGNOS_TO_LOOKER, inplace=True)
print(df_cognos.columns)
print(df_looker.columns)

Index(['S/A OPUS Month', '2018/M03', '2018/M04', '2018/M05', '2018/M06',
       '2018/M07', '2018/M08', '2018/M09', '2018/M10', '2018/M11', '2018/M12',
       '2019/M01', '2019/M02', '2019/M03', '2019/M04', '2019/M05', '2019/M06',
       '2019/M07', '2019/M08', '2019/M09', '2019/M10', '2019/M11', '2019/M12',
       '2020/M01', '2020/M02', '2020/M03', '2020/M04', '2020/M05', '2020/M06',
       '2020/M07', '2020/M08', '2020/M09', '2020/M10', '2020/M11', '2020/M12',
       '2021/M01', '2021/M02', '2021/M03', '2021/M04', '2021/M05', '2021/M06',
       '2021/M07', '2021/M08', '2021/M09', '2021/M10', '2021/M11', '2021/M12',
       '2022/M01', '2022/M02', '2022/M03', '2022/M04', '2022/M05', '2022/M06',
       '2022/M07', '2022/M08', '2022/M09', '2022/M10', '2022/M11', '2022/M12',
       '2023/M01', '2023/M02', '2023/M03', '2023/M04', '2023/M05', '2023/M06',
       '2023/M07', '2023/M08', '2023/M09', '2023/M10', '2023/M11', '2023/M12'],
      dtype='object')
Index(['S/A OPUS Month', '2018/M03'

In [83]:
df_cognos = df_cognos.drop(columns=['2022/M05', '2022/M06',
  '2022/M07', '2022/M08', '2022/M09', '2022/M10', '2022/M11', '2022/M12',
  '2023/M01', '2023/M02', '2023/M03', '2023/M04', '2023/M05', '2023/M06',
  '2023/M07', '2023/M08', '2023/M09', '2023/M10', '2023/M11', '2023/M12'], axis=1)

for column in  [x for x in df_looker.columns if x !='S/A OPUS Month']:
  if df_looker[column].dtype == 'object':
    df_looker[column].fillna("No_data", inplace=True)
    df_cognos[column].fillna("No_data", inplace=True)
  if df_looker[column].dtype == 'datetime64[ns]':
    df_cognos[column] = df_cognos.apply(lambda row:
      pd.Timestamp(row[column]).round('S'), axis=1)
    df_looker[column] = df_looker.apply(lambda row:
      pd.Timestamp(row[column]).round('S'), axis=1)
    df_looker[column].fillna(0, inplace=True)
    df_cognos[column].fillna(0, inplace=True)
  if df_looker[column].dtype == 'float64' or df_looker[column].dtype == 'int64':
    df_looker[column].fillna(0, inplace=True)
    df_cognos[column].fillna(0, inplace=True)
df_cognos

,S/A OPUS Month,2018/M03,2018/M04,2018/M05,2018/M06,2018/M07,2018/M08,2018/M09,2018/M10,2018/M11,...,2021/M07,2021/M08,2021/M09,2021/M10,2021/M11,2021/M12,2022/M01,2022/M02,2022/M03,2022/M04
0,EAS,5265.54,197716077.82,288981699.83,313192561.76,325239414.36,331404289.04,342665630.98,329834821.44,341648362.68,...,966777160.96,1011394551.88,1002927311.91,1089304837.43,1060087476.2,1127640900.38,1191286321.77,891923068.48,1031711433.42,1053616323.43
1,EUA,No_data,62515902.17,148851911.27,176986262.7,198138474.72,205514292.16,215789966.13,221684933.41,189019369.16,...,523867877.78,582273041,608331782.14,586394222.58,600109065.91,670238923.23,675110992.92,654157082.43,702723736.33,655609272.81
2,LA,No_data,13145264.82,30121801.05,35729513.48,41313911.16,47439759.26,48121062.41,49932310.38,47508493.2,...,125066457.16,135266249.69,131618513.12,144663591.18,172699982.44,191323276.8,185541501.41,166498328.13,178192866.53,141021120.96
3,NaN,No_data,72910133.91,238606996.51,256684126.68,286898275.63,318772930.31,314757359.6,330086412.15,318801143.61,...,555132286.98,615308550.05,613692822.78,669536583.7,533113356.11,568244201.94,561476910.21,522858610.53,630682066.41,542523491.8
4,SAS,82301.88,66080467.39,111839777.97,116865672.59,129404402.06,133200801.25,126287042.32,135845133.45,134067508.45,...,277877498.76,292792829.39,317186918.01,339597263.68,325675691.73,360442289.82,367930238.4,356074820.74,391809885.58,373457471.88


In [84]:
df_looker

,S/A OPUS Month,2018/M03,2018/M04,2018/M05,2018/M06,2018/M07,2018/M08,2018/M09,2018/M10,2018/M11,...,2021/M07,2021/M08,2021/M09,2021/M10,2021/M11,2021/M12,2022/M01,2022/M02,2022/M03,2022/M04
0,SAS,82301.88,66080467.39,111839777.97,116865672.59,129404402.06,133200801.25,126287042.32,135845133.45,134067508.45,...,277877498.76,292792829.39,317186918.01,339597263.68,325675691.73,360442289.82,367930238.4,356074820.74,391809885.58,373457471.88
1,EAS,5265.54,197716077.82,288981699.83,313192561.76,325239414.36,331404289.04,342665630.98,329834821.44,341648362.68,...,966777160.96,1011394551.88,1002927311.91,1089304837.43,1060087476.2,1127640900.38,1191286321.77,891923068.48,1031711433.42,1053616323.43
2,NaN,No_data,72910133.91,238606996.51,256684126.68,286898275.63,318772930.31,314757359.6,330086412.15,318801143.61,...,555132286.98,615308550.05,613690182.78,669536583.7,533113356.11,568244201.94,561476910.21,522858160.53,630682066.41,542501591.8
3,LA,No_data,13145264.82,30121801.05,35729513.48,41313911.16,47439759.26,48121062.41,49932310.38,47508493.2,...,125066457.16,135266249.69,131618513.12,144662541.18,172700782.44,191323276.8,185541501.41,166498868.13,178192866.53,141021120.96
4,EUA,No_data,62515902.17,148851911.27,176986262.7,198138474.72,205514292.16,215789966.13,221684933.41,189019369.16,...,523867877.78,582273041,608331782.14,586394222.58,600109065.91,670238923.23,675110992.92,654157082.43,702724256.98,655610561.21


In [85]:
# compare = datacompy.Compare( df_cognos, df_looker, join_columns=['S/A OPUS Month'], df1_name='Cognos', df2_name='Looker')
# print(compare.report())

In [86]:
# Hiển thị danh sách column và số lượng column
def get_column_lists_and_counts(df_cognos, df_looker):
  columns_df_cognos = df_cognos.columns.tolist()
  columns_df_looker = df_looker.columns.tolist()
  num_columns_df_cognos = len(columns_df_cognos)
  num_columns_df_looker = len(columns_df_looker)
  print("Columns in df_cognos:", columns_df_cognos)
  print("Number of columns in df_cognos:", num_columns_df_cognos)
  print("Columns in df_looker:", columns_df_looker)
  print("Number of columns in df_looker:", num_columns_df_looker)
  return columns_df_cognos, num_columns_df_cognos, columns_df_looker, num_columns_df_looker

# Định nghĩa hàm để xóa các cột từ DataFrame
def drop_columns(df, columns_to_drop):
  df = df.drop(columns=columns_to_drop)
  return df

# Hàm Đổi các cột có nhãn (Ví dụ: Y -> Yes, N -> No)
def replace_values(df, columns, replacement):
  for column in columns:
    df[column] = df[column].replace(replacement)
  return df

#Định nghĩa hàm xóa dấu phẩy và convert cột sang kiểu Float để so sánh các cột số
def replace_comma_and_convert_to_float(df, column_names):
  for column_name in column_names:
    if column_name in df.columns:  # Kiểm tra xem cột có tồn tại trong DataFrame không
      if df[column_name].dtype == 'object' and df[column_name].str.contains(',').any():
        df[column_name] = df[column_name].str.replace(',', '', regex=True)
      df[column_name] = df[column_name].astype(float)


# Convert sang datetime
def convert_columns_to_datetime(df, columns):
    for column in columns:
        try:
            df[column] = pd.to_datetime(df[column], errors='coerce')
        except (KeyError, ValueError):
            print(f"Column '{column}' does not exist or cannot be converted to datetime.")

# fillna cho các cột chuỗi và số
def fillna_custom(df, string_columns, numeric_columns):
    for col in string_columns:
        if col in df.columns:
            df[col] = df[col].fillna("No_Data")
    for col in numeric_columns:
        if col in df.columns:
            df[col] = df[col].fillna(0)
    return df

#Định nghĩa hàm trả về kiểu dữ liệu các column của DF
def check_and_print_column_data_types(df, name):
    data_types_df = df.dtypes
    print(f"Data types for DataFrame {name}:")
    print(data_types_df)

# KIỂM TRA CÁC COLUMN ĐÃ CHỌN CÓ TẠO RA ĐƯỢC TỔ HỢP KHÓA HAY KHÔNG
def check_columns_combination_uniqueness(df_cognos, df_looker, columns_to_check):
    def combine_columns(df):
        return df[columns_to_check].apply(lambda x: '-'.join(x.astype(str).str.upper()), axis=1)
    df_cognos_combined = combine_columns(df_cognos)
    df_looker_combined = combine_columns(df_looker)
    are_columns_combined_unique = df_cognos_combined.is_unique and df_looker_combined.is_unique
    if are_columns_combined_unique:
        result = True
    else:
        result = False
    return result

#ĐỊNH NGHĨA HÀM ĐỂ CHUYỂN SANG CHỮ IN HOA
def uppercase_columns(df, columns):
    for col in columns:
        if col in df.columns:
            df[col] = df[col].str.upper()
    return df

# ĐỊNH NGHĨA HÀM: TÌM VÀ CẮT DUOID (VD: _cognos, _looker)
def find_and_strip_columns_with_suffix(df, suffix):
    columns_with_suffix = [col for col in df.columns if col.endswith(suffix)]
    stripped_columns = [col.replace(suffix, '') for col in columns_with_suffix]
    return stripped_columns

# ĐỊNH NGHĨA HÀM: TÌM RA SỰ KHÁC BIỆT CỦA 2 DF (MISSING, DIFF) RÒI FILLNA
# Tìm sự khác biẹt
def create_diff_df(df_merged, columns):
    conditions = []
    for column in columns:
        data_cognos = df_merged[f"{column}_cognos"]
        data_looker = df_merged[f"{column}_looker"]
        if data_cognos.dtype== np.float64:
            data_cognos = data_cognos.round(2)
            data_looker = data_looker.round(2)
            condition = np.isclose(data_cognos, data_looker, atol=0.1)
        elif data_cognos.dtype== np.datetime64:
            rounded_cognos = data_cognos.dt.round('S')
            rounded_looker = data_looker.dt.round('S')
            condition = (rounded_cognos == rounded_looker)
        elif data_cognos.dtype== object:
            condition = (data_cognos.str.upper() == data_looker.str.upper())
        else:
            continue
        conditions.append(condition)
    combined_condition = np.all(conditions, axis=0)
    diff_df = df_merged[~combined_condition].copy()
    matching_df = df_merged[combined_condition].copy()
    diff_df = diff_df.applymap(lambda x: 'Not exists' if pd.isnull(x) else x)
    matching_df = matching_df.applymap(lambda x: 'Not exists' if pd.isnull(x) else x)
    return matching_df, diff_df


# LỌC RA PHẦN MISSING HAY DIFF
def filter_and_return_dataframes(df):
    df_missing_cognos = df.loc[df.filter(like='_cognos').eq('Not exists').all(axis=1)]
    df_missing_looker = df.loc[df.filter(like='_looker').eq('Not exists').all(axis=1)]
    df_remaining_diff = df.drop(df_missing_cognos.index).drop(df_missing_looker.index)
    return df_missing_cognos, df_missing_looker, df_remaining_diff.reset_index()

# TÁCH DF CỦA COGNOS VÀ LOOKER
def split_dataframe(df):
    cols_df_cognos = [col for col in df.columns if not col.endswith("_looker")]
    df_cognos_new = df[cols_df_cognos]
    cols_df_looker = [col for col in df.columns if not col.endswith("_cognos")]
    df_looker_new = df[cols_df_looker]
    return df_cognos_new, df_looker_new

# COMPARE VÀ LÊN MÀU CHO SỰ KHÁC BIẸT
def compare_and_style_data(diff_df, keys, suffix_cognos, suffix_looker):
    def highlight_cells(val):
        if val == 'Same':
            return ''
        else:
            return 'background-color: red; color: white;'
    common_prefixes = set(col.split(suffix_cognos)[0] for col in diff_df.columns[2:])
    for prefix in common_prefixes:
        cognos_cols = [col for col in diff_df.columns if col.startswith(f'{prefix}{suffix_cognos}')]
        looker_cols = [col for col in diff_df.columns if col.startswith(f'{prefix}{suffix_looker}')]
        for cognos_col, looker_col in zip(cognos_cols, looker_cols):
            column_name = f'Compare_{prefix}_(Cognos <> Looker)'
            data_type = diff_df[cognos_col].dtype
            if data_type == np.float64:
                diff_df[cognos_col] = diff_df[cognos_col].round(2)
                diff_df[looker_col] = diff_df[looker_col].round(2)
                condition = np.isclose(diff_df[cognos_col], diff_df[looker_col], atol=0.1)
            elif data_type == np.datetime64:
                rounded_cognos = diff_df[cognos_col].dt.round('S')
                rounded_looker = diff_df[looker_col].dt.round('S')
                condition = (rounded_cognos == rounded_looker)
            elif data_type == object:
                condition = (diff_df[cognos_col].str.upper() == diff_df[looker_col].str.upper())
            else:
                continue
            diff_df[column_name] = diff_df.apply(lambda row: f'({row[cognos_col]} <> {row[looker_col]})'
                                                if not condition[row.name]
                                                else 'Same', axis=1)
    selected_keys = keys + [col for col in diff_df.columns if col.startswith('Compare')]
    styled_df = diff_df[selected_keys].style.applymap(highlight_cells, subset=pd.IndexSlice[:, diff_df[selected_keys].columns.str.startswith('Compare_')])
    return styled_df


# ĐỊNH NGHĨA HÀM TÍNH TOÁN
def total_numeric_columns(df_Cognos_cal, df_Looker_cal, columns_to_compare, sales_week_col, start_week=None, end_week=None):
    df_Looker_cal['Source'] = 'Looker'
    df_Cognos_cal['Source'] = 'Cognos'
    combined_data = pd.concat([df_Cognos_cal, df_Looker_cal])
    if start_week and end_week:
        filtered_data = combined_data[(combined_data[sales_week_col] >= start_week) & (combined_data[sales_week_col] <= end_week)]
    else:
        filtered_data = combined_data
    filtered_data[columns_to_compare] = filtered_data[columns_to_compare].round(2)
    result = filtered_data.pivot_table(index=[sales_week_col], columns='Source', values=columns_to_compare, aggfunc='sum')
    for column in columns_to_compare:
        result[f'Compare_{column}_(Cognos <> Looker)'] = result.apply(lambda row: 'Same' if (np.isclose(row[column]['Cognos'], row[column]['Looker'], atol=0.1) == True) else f"{row[column]['Cognos']} <> {row[column]['Looker']}", axis=1)
    result = result.swaplevel(axis=1).sort_index(axis=1, level=[1, 0], ascending=[True, False])
    return result

# ĐỊNH NGHĨA HÀM ĐỂ ĐẶT LẠI TÊN CŨ TRƯỚC KHI XUẤT FILE
# Định nghĩa hàm clean_and_reorder_columns nếu chưa có
def clean_and_reorder_columns(df, suffix_to_remove, new_column_order, new_column_names, output_file):
    df.columns = [col.replace(suffix_to_remove, '') for col in df.columns]
    df = df[new_column_order]
    df.rename(columns=dict(zip(df.columns, new_column_names)), inplace=True)
    df.to_csv(os.path.join(folder_path, output_file), index=False)
    return df

In [87]:
keys_list=['S/A OPUS Month']
REMOVE_COLUMN = [x for x in list(df_looker.columns) if x not in keys_list]

df_merged = pd.merge(df_cognos, df_looker,  on=keys_list, how='outer', suffixes=('_cognos', '_looker'))
matching_df, diff_df = create_diff_df(df_merged, REMOVE_COLUMN)

df_missing_cognos, df_missing_looker, df_remaining_diff = filter_and_return_dataframes(diff_df)
df_cognos_diff_new, df_looker_diff_new = split_dataframe(df_remaining_diff)
df_not_in_cognos, df_only_in_looker = split_dataframe(df_missing_cognos)
df_only_in_cognos, df_not_in_looker = split_dataframe(df_missing_looker)
different_df = df_remaining_diff
selected_keys = keys_list
styled_result = compare_and_style_data(different_df, selected_keys, '_cognos', '_looker')
styled_result.to_excel(os.path.join(r'C:\Users\hoa.nd\Desktop\WAP\compare\Compare report\report', 'AR History Region.xlsx'), index=False)
styled_result  

,S/A OPUS Month,Compare_2020/M07_(Cognos <> Looker),Compare_2021/M03_(Cognos <> Looker),Compare_2021/M12_(Cognos <> Looker),Compare_2020/M11_(Cognos <> Looker),Compare_2020/M01_(Cognos <> Looker),Compare_2022/M02_(Cognos <> Looker),Compare_2021/M05_(Cognos <> Looker),Compare_2020/M03_(Cognos <> Looker),Compare_2018/M06_(Cognos <> Looker),Compare_2019/M02_(Cognos <> Looker),Compare_2021/M11_(Cognos <> Looker),Compare_2018/M10_(Cognos <> Looker),Compare_2020/M08_(Cognos <> Looker),Compare_2021/M02_(Cognos <> Looker),Compare_2018/M05_(Cognos <> Looker),Compare_2018/M07_(Cognos <> Looker),Compare_2021/M04_(Cognos <> Looker),Compare_2020/M10_(Cognos <> Looker),Compare_2021/M06_(Cognos <> Looker),Compare_2020/M09_(Cognos <> Looker),Compare_2019/M07_(Cognos <> Looker),Compare_2019/M08_(Cognos <> Looker),Compare_2019/M06_(Cognos <> Looker),Compare_2018/M08_(Cognos <> Looker),Compare_2018/M12_(Cognos <> Looker),Compare_2018/M09_(Cognos <> Looker),Compare_2022/M03_(Cognos <> Looker),Compare_2018/M03_(Cognos <> Looker),Compare_2018/M11_(Cognos <> Looker),Compare_2020/M06_(Cognos <> Looker),Compare_2021/M07_(Cognos <> Looker),Compare_2022/M04_(Cognos <> Looker),Compare_2019/M04_(Cognos <> Looker),Compare_2021/M10_(Cognos <> Looker),Compare_2019/M05_(Cognos <> Looker),Compare_2019/M01_(Cognos <> Looker),Compare_2019/M10_(Cognos <> Looker),Compare_2021/M08_(Cognos <> Looker),Compare_2020/M04_(Cognos <> Looker),Compare_2019/M09_(Cognos <> Looker),Compare_2021/M01_(Cognos <> Looker),Compare_2018/M04_(Cognos <> Looker),Compare_2019/M03_(Cognos <> Looker),Compare_2020/M12_(Cognos <> Looker),Compare_2022/M01_(Cognos <> Looker),Compare_2019/M12_(Cognos <> Looker),Compare_2020/M05_(Cognos <> Looker),Compare_2021/M09_(Cognos <> Looker),Compare_2020/M02_(Cognos <> Looker),Compare_2019/M11_(Cognos <> Looker)
0,EAS,Same,Same,Same,Same,Same,Same,(797096288.1 <> 797262238.91),Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same
1,EUA,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,(443967002.91 <> 443982646.68),Same,Same,Same,Same,Same,Same,Same,(702723736.33 <> 702724256.98),Same,Same,Same,Same,(655609272.81 <> 655610561.21),Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same
2,LA,Same,(111983907.56 <> 111984017.56),Same,Same,Same,(166498328.13 <> 166498868.13),Same,Same,Same,Same,(172699982.44 <> 172700782.44),Same,Same,Same,Same,Same,(97716942.52 <> 97717602.52),Same,(110466499.52 <> 110466999.52),Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,(144663591.18 <> 144662541.18),Same,Same,Same,Same,Same,Same,(69631854.15 <> 69636054.15),Same,Same,Same,Same,Same,Same,Same,Same,Same
3,Not exists,Same,(456088547.86 <> 456087007.86),Same,Same,Same,(522858610.53 <> 522858160.53),Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,(542523491.8 <> 542501591.8),Same,Same,Same,Same,Same,Same,(309242902.18 <> 309242482.18),Same,Same,Same,Same,Same,Same,Same,Same,(613692822.78 <> 613690182.78),Same,Same
4,SAS,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,(141784605.12 <> 141803840.76),Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same,Same
